In [1]:
!git clone https://github.com/google-research/bert

fatal: destination path 'bert' already exists and is not an empty directory.


In [5]:
!gsutil cp gs://bucket_armbert/hy.txt .

Copying gs://bucket_armbert/hy.txt...
| [1 files][ 12.6 GiB/ 12.6 GiB]   67.7 MiB/s                                   
Operation completed over 1 objects/12.6 GiB.                                     


In [6]:
!mkdir ./shards
!split -a 4 -l 256000 -d 'hy.txt' ./shards/
!ls ./shards/

mkdir: cannot create directory ‘./shards’: File exists
^C
0000  0004  0008  0012	0016  0020  0024  0028	0032  0036  0040  0161
0001  0005  0009  0013	0017  0021  0025  0029	0033  0037  0041
0002  0006  0010  0014	0018  0022  0026  0030	0034  0038  0042
0003  0007  0011  0015	0019  0023  0027  0031	0035  0039  0160


In [1]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = False #@param {type:"boolean"}
PROCESSES = 1 #@param {type:"integer"}
VOC_FNAME = 'armenian-vocab.txt'
PRETRAINING_DIR = "./Data" #@param {type:"string"}

In [2]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD

INFO:tensorflow:*** Reading from input files ***
I0605 14:04:47.226516 139963333158720 create_pretraining_data.py:446] *** Reading from input files ***
INFO:tensorflow:  ./shards/0000
I0605 14:04:47.226745 139963333158720 create_pretraining_data.py:448]   ./shards/0000


In [3]:
BUCKET_NAME = "bucket_armbert" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)

if not BUCKET_NAME:
    log.warning("WARNING: BUCKET_NAME is not set. "
              "You will not be able to train the model.")

In [4]:
with open('armenian-vocab.txt', 'r') as f:
    bert_vocab = f.read().split('\n')

In [5]:
import json
# use this for BERT-base

bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": 64000
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
    json.dump(bert_base_config, fo, indent=2)

with open("{}/{}".format(MODEL_DIR, 'armenian-vocab.txt'), "w") as fo:
    for token in bert_vocab:
        fo.write(token + "\n")

In [6]:
!echo $BUCKET_NAME

bucket_armbert


In [7]:
if BUCKET_NAME:
    !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

Copying file://bert_model/armenian-vocab.txt [Content-Type=text/plain]...
Copying file://bert_model/bert_config.json [Content-Type=application/json]...   
Copying file://./Data/0017.tfrecord [Content-Type=application/octet-stream]...  
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://./Data/0083.tfrecord [Content-Type=application/octet-stream]...

In [2]:
import os
import logging
import tensorflow
import json

# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'XRT_TPU_CONFIG' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['XRT_TPU_CONFIG'].split(';')[2]

    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        # Upload credentials to TPU.
        with open('armeniannlp-314719-f051f32c157e.json', 'r') as f:
            auth_info = json.load(f)
        tensorflow.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

2021-06-12 08:26:01,196 :  Using TPU runtime
2021-06-12 08:26:01,214 :  TPU address is grpc://10.17.71.218:8470
2021-06-12 08:26:01,216 :  
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
from bert import modeling

BUCKET_NAME = "bucket_armbert" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
PRETRAINING_DIR = "Data" #@param {type:"string"}
VOC_FNAME = "armenian-vocab.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
    BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
    BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2021-06-12 08:26:05,257 :  Using checkpoint: gs://bucket_armbert/bert_model/model.ckpt-1000000
2021-06-12 08:26:05,258 :  Using 159 data shards


In [4]:
from bert.run_pretraining import input_fn_builder, model_fn_builder

model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tensorflow.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tensorflow.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tensorflow.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tensorflow.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tensorflow.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

2021-06-12 08:26:08,015 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f7699040560>) includes params argument, but params are not passed to Estimator.
2021-06-12 08:26:08,017 :  Using config: {'_model_dir': 'gs://bucket_armbert/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.17.71.218:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7699052f50>, '_

In [5]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2021-06-12 08:26:12,845 :  Skipping training since max_steps has already saved.
2021-06-12 08:26:12,846 :  training_loop marked as finished
